# 📘 Архитектуры Агентов 8: Стек эпизодической и семантической памяти

Добро пожаловать в восьмой блокнот нашей серии. Сегодня мы решим одну из самых критических задач в создании по-настоящему интеллектуальных, долгосрочных помощников: **постоянная память (persistent memory)**. Память стандартного чат-бота эфемерна и длится только одну сессию. Чтобы создать персонализированного агента, который учится и растет вместе с пользователем, нам нужно более надежное решение.

Мы реализуем структурированную архитектуру памяти, которая зеркально отражает человеческое познание, объединяя два различных типа памяти:

1.  **Episodic Memory (Эпизодическая память):** Это память о конкретных событиях или прошлых взаимодействиях. Она отвечает на вопрос "Что произошло?" (например, "На прошлой неделе пользователь спрашивал меня о цене акций NVIDIA."). Для поиска прошлых разговоров, релевантных текущей теме, мы будем использовать **векторную базу данных (vector database)**.
2.  **Semantic Memory (Семантическая память):** Это память о структурированных фактах, концепциях и отношениях, извлеченных из этих событий. Она отвечает на вопрос "Что я знаю?" (например, "Пользователь Алекс — консервативный инвестор.", "Алекс интересуется технологическими акциями."). Для этого мы будем использовать **графовую базу данных (graph database)** (Neo4j), так как она превосходно справляется с управлением и запросами сложных связей.

Объединяя их, наш агент может не только вспоминать прошлые разговоры, но и строить богатую, взаимосвязанную базу знаний о пользователе и мире, что ведет к глубоко персонализированным и контекстно-зависимым взаимодействиям.

### Определение
**Episodic + Semantic Memory Stack** — это архитектура агента, которая поддерживает два типа долгосрочной памяти. **Episodic memory** (эпизодическая память) хранит хронологический журнал опыта (например, краткие содержания чатов) и обычно ищется на основе семантического сходства. **Semantic memory** (семантическая память) хранит извлеченные структурированные знания (факты, сущности, отношения) в базе знаний, часто в виде графа.

### Высокоуровневый рабочий процесс

1.  **Взаимодействие (Interaction):** Агент ведет диалог с пользователем.
2.  **Извлечение из памяти (Memory Retrieval / Recall):** Для нового запроса пользователя агент сначала опрашивает обе системы памяти.
    *   Он ищет в **эпизодическом** векторном хранилище похожие прошлые разговоры.
    *   Он запрашивает в **семантической** базе данных графа сущности и факты, связанные с запросом.
3.  **Дополненная генерация (Augmented Generation):** Извлеченные воспоминания добавляются в контекст промпта, позволяя LLM генерировать ответ, который учитывает прошлые взаимодействия и изученные факты.
4.  **Создание памяти (Memory Creation / Encoding):** После завершения взаимодействия фоновый процесс анализирует разговор.
    *   Он создает краткое резюме обмена репликами (новая **эпизодическая** память).
    *   Он извлекает ключевые сущности и отношения (новая **семантическая** память).
5.  **Хранение памяти (Memory Storage):** Новое эпизодическое резюме векторизуется (embedded) и сохраняется в векторное хранилище. Новые семантические факты записываются как узлы и ребра в графовую базу данных.

### Когда использовать / Применения
*   **Долгосрочные персональные ассистенты:** Ассистент, который помнит ваши предпочтения, проекты и личные детали на протяжении недель или месяцев.
*   **Персонализированные системы:** Боты электронной коммерции, которые помнят ваш стиль, или образовательные тьюторы, которые помнят ваш прогресс обучения и слабые места.
*   **Сложные исследовательские агенты:** Агент, который строит граф знаний по теме по мере изучения документов, что позволяет ему отвечать на сложные, многоходовые вопросы.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Истинная персонализация:** Обеспечивает контекст и обучение, которые сохраняются бесконечно, далеко за пределами контекстного окна одной сессии.
    *   **Богатое понимание:** Графовая база данных позволяет агенту понимать и рассуждать о сложных взаимосвязях между сущностями.
*   **Слабые стороны:**
    *   **Сложность:** Эту архитектуру значительно сложнее построить и поддерживать, чем простого агента без состояния (stateless).
    *   **Разрастание памяти и очистка (Pruning):** Со временем хранилища памяти могут стать массивными. Стратегии для суммирования, консолидации или очистки старых/нерелевантных воспоминаний необходимы для долгосрочной производительности.

## Фаза 0: Основа и Настройка

Мы установим все необходимые библиотеки, включая драйверы для наших векторных и графовых баз данных, и настроим наши API ключи.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain_community langchain-openai neo4j faiss-cpu tiktoken

In [ ]:
import os
import uuid
from typing import List, Dict, Any, Optional, Tuple

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius, NebiusEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Memory Stack (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"]
)

Environment variables loaded and tracing is set up.


## Фаза 1: Построение Компонентов Памяти

Это ядро нашей архитектуры. Мы определим структуры для наших воспоминаний и настроим соединения с базами данных. Мы также создадим агента "Memory Maker" (Создатель памяти), ответственного за обработку разговоров и создание новых воспоминаний.

In [ ]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
embeddings = NebiusEmbeddings()

# --- 1. Векторное хранилище для Эпизодической памяти ---
# В реальном приложении вы бы сохраняли это на диск. В этом примере оно находится в памяти.
try:
    episodic_vector_store = FAISS.from_texts(["Initial document to bootstrap the store"], embeddings)
except ImportError:
    console.print("[bold red]FAISS not installed. Please run `pip install faiss-cpu`.")[/bold red]
    episodic_vector_store = None

# --- 2. Графовая БД для Семантической памяти ---
try:
    graph = Neo4jGraph(
        url=os.environ.get("NEO4J_URI"),
        username=os.environ.get("NEO4J_USERNAME"),
        password=os.environ.get("NEO4J_PASSWORD")
    )
    # Очистка графа для чистого запуска
    graph.query("MATCH (n) DETACH DELETE n")
except Exception as e:
    console.print(f"[bold red]Failed to connect to Neo4j: {e}. Please check your credentials and connection.[/bold red]")
    graph = None

# --- 3. Pydantic модели для "Memory Maker" ---
# Определение структуры знаний, которые мы хотим извлечь.
class Node(BaseModel):
    id: str = Field(description="Unique identifier for the node, which can be a person's name, a company ticker, or a concept.")
    type: str = Field(description="The type of the node (e.g., 'User', 'Company', 'InvestmentPhilosophy').")
    properties: Dict[str, Any] = Field(description="A dictionary of properties for the node.")

class Relationship(BaseModel):
    source: Node = Field(description="The source node of the relationship.")
    target: Node = Field(description="The target node of the relationship.")
    type: str = Field(description="The type of the relationship (e.g., 'IS_A', 'INTERESTED_IN').")
    properties: Dict[str, Any] = Field(description="A dictionary of properties for the relationship.")

class KnowledgeGraph(BaseModel):
    """Представляет структурированные знания, извлеченные из разговора."""
    relationships: List[Relationship] = Field(description="A list of relationships to be added to the knowledge graph.")

# --- 4. Агент "Memory Maker" ---
def create_memories(user_input: str, assistant_output: str):
    conversation = f"User: {user_input}\nAssistant: {assistant_output}"
    
    # 4a. Создание Эпизодической памяти (Суммаризация)
    console.print("--- Создание Эпизодической памяти (Summary) ---")
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a summarization expert. Create a concise, one-sentence summary of the following user-assistant interaction. This summary will be used as a memory for future recall."),
        ("human", "Interaction:\n{interaction}")
    ])
    summarizer = summary_prompt | llm
    episodic_summary = summarizer.invoke({"interaction": conversation}).content
    
    new_doc = Document(page_content=episodic_summary, metadata={"created_at": uuid.uuid4().hex})
    episodic_vector_store.add_documents([new_doc])
    console.print(f"[green]Эпизодическая память создана:[/green] '{episodic_summary}'")
    
    # 4b. Создание Семантической памяти (Извлечение фактов)
    console.print("--- Создание Семантической памяти (Graph) ---")
    extraction_llm = llm.with_structured_output(KnowledgeGraph)
    extraction_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a knowledge extraction expert. Your task is to identify key entities and their relationships from a conversation and model them as a graph. Focus on user preferences, goals, and stated facts."),
        ("human", "Extract all relationships from this interaction:\n{interaction}")
    ])
    extractor = extraction_prompt | extraction_llm
    try:
        kg_data = extractor.invoke({"interaction": conversation})
        if kg_data.relationships:
            for rel in kg_data.relationships:
                graph.add_graph_documents([rel], include_source=True)
            console.print(f"[green]Семантическая память создана:[/green] Добавлено {len(kg_data.relationships)} отношений в граф.")
        else:
            console.print("[yellow]Новые семантические воспоминания в этом взаимодействии не обнаружены.[/yellow]")
    except Exception as e:
        console.print(f"[red]Не удалось извлечь или сохранить семантическую память: {e}[/red]")

if episodic_vector_store and graph:
    print("Компоненты памяти инициализированы успешно.")

Memory components initialized successfully.


## Фаза 2: Агент с дополненной памятью

Теперь мы создадим агента, который использует эту систему памяти. Мы будем использовать LangGraph для определения четкого рабочего процесса с состоянием: извлечь воспоминания, сгенерировать ответ, используя эти воспоминания, и, наконец, обновить память последним взаимодействием.

In [ ]:
# Определение состояния для нашего агента LangGraph
class AgentState(TypedDict):
    user_input: str
    retrieved_memories: Optional[str]
    generation: str

# Определение узлов графа

def retrieve_memory(state: AgentState) -> Dict[str, Any]:
    """Узел, который извлекает воспоминания как из эпизодического, так и из семантического хранилищ."""
    console.print("--- Извлечение Воспоминаний ---")
    user_input = state['user_input']
    
    # Извлечение из эпизодической памяти
    retrieved_docs = episodic_vector_store.similarity_search(user_input, k=2)
    episodic_memories = "\n".join([doc.page_content for doc in retrieved_docs])
    
    # Извлечение из семантической памяти
    # Это простое извлечение; более продвинутое включало бы извлечение сущностей из запроса
    try:
        graph_schema = graph.get_schema
        # Использование полнотекстового индекса для лучшего поиска. Neo4j автоматически создает его на свойствах узлов.
        # Более надежное решение может включать предварительное извлечение сущностей из user_input.
        semantic_memories = str(graph.query("""
            UNWIND $keywords AS keyword
            CALL db.index.fulltext.queryNodes("entity", keyword) YIELD node, score
            MATCH (node)-[r]-(related_node)
            RETURN node, r, related_node LIMIT 5
            """, {'keywords': user_input.split()}))
    except Exception as e:
        semantic_memories = f"Could not query graph: {e}"
        
    retrieved_content = f"Relevant Past Conversations (Episodic Memory):\n{episodic_memories}\n\nRelevant Facts (Semantic Memory):\n{semantic_memories}"
    console.print(f"[cyan]Извлеченный контекст:\n{retrieved_content}[/cyan]")
    
    return {"retrieved_memories": retrieved_content}

def generate_response(state: AgentState) -> Dict[str, Any]:
    """Узел, который генерирует ответ, используя извлеченные воспоминания."""
    console.print("--- Генерация Ответа ---")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful and personalized financial assistant. Use the retrieved memories to inform your response and tailor it to the user. If the memories indicate a user's preference (e.g., they are a conservative investor), you MUST respect it."),
        ("human", "My question is: {user_input}\n\nHere are some memories that might be relevant:\n{retrieved_memories}")
    ])
    generator = prompt | llm
    generation = generator.invoke(state).content
    console.print(f"[green]Сгенерированный ответ:\n{generation}[/green]")
    return {"generation": generation}

def update_memory(state: AgentState) -> Dict[str, Any]:
    """Узел, который обновляет память последним взаимодействием."""
    console.print("--- Обновление Памяти ---")
    create_memories(state['user_input'], state['generation'])
    return {}

# Построение графа
workflow = StateGraph(AgentState)

workflow.add_node("retrieve", retrieve_memory)
workflow.add_node("generate", generate_response)
workflow.add_node("update", update_memory)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", "update")
workflow.add_edge("update", END)

memory_agent = workflow.compile()
print("Граф агента с дополненной памятью успешно скомпилирован.")

Memory-augmented agent graph compiled successfully.


## Фаза 3: Демонстрация и Инспекция

Давайте посмотрим на агента в действии. Мы просимулируем многоходовую беседу. Первые два хода будут "сеять" память. Третий ход проверит, может ли агент использовать эту память для персонализированного ответа. Наконец, мы напрямую проверим базы данных, чтобы увидеть созданные воспоминания.

In [ ]:
def run_interaction(query: str):
    result = memory_agent.invoke({"user_input": query})
    return result['generation']

console.print("\n--- 💬 ВЗАИМОДЕЙСТВИЕ 1: Засев Памяти (Seeding) ---")
run_interaction("Hi, my name is Alex. I'm a conservative investor, and I'm mainly interested in established tech companies.")

console.print("\n--- 💬 ВЗАИМОДЕЙСТВИЕ 2: Задание конкретного вопроса ---")
run_interaction("What do you think about Apple (AAPL)?")

console.print("\n--- 🧠 ВЗАИМОДЕЙСТВИЕ 3: ТЕСТ ПАМЯТИ ---")
run_interaction("Based on my goals, what's a good alternative to that stock?")


--- 💬 INTERACTION 1: Seeding Memory ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
Initial document to bootstrap the store

Relevant Facts (Semantic Memory):
[]
--- Generating Response ---
Generated Response:
Hello, Alex! It's great to meet you. As a conservative investor, focusing on established tech companies with strong fundamentals is a very sound strategy. I can certainly help you navigate that space. What's on your mind today?
--- Updating Memory ---
--- Creating Episodic Memory (Summary) ---
Episodic memory created: 'The user, Alex, introduced himself as a conservative investor interested in established tech companies.'
--- Creating Semantic Memory (Graph) ---
Semantic memory created: Added 2 relationships to the graph.



--- 💬 INTERACTION 2: Asking a specific question ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
The user, Alex, introduced himself as a conservative investor interested in established tech companies.
Initial document to bootstrap the store

Relevant Facts (Semantic Memory):
[]
--- Generating Response ---
Generated Response:
Apple (AAPL) is often considered a cornerstone for conservative tech portfolios. It has a massive market capitalization, a very strong brand, consistent profitability, and a history of returning value to shareholders through dividends and buybacks. Its ecosystem of products creates a loyal customer base, which provides a stable revenue stream. For a conservative investor, it generally aligns well with the goal of capital preservation while still offering growth potential. Would you like a deeper dive into its recent performance or financials?
--- Updating Memory ---
--- Creating Episodic Memory (Summary) ---
Episodic memory created: 'The user inquired about Apple (AAP


--- 🧠 INTERACTION 3: THE MEMORY TEST ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
The user, Alex, introduced himself as a conservative investor interested in established tech companies.
The user inquired about Apple (AAPL), and the assistant confirmed it's a suitable stock for conservative investors due to its stability and market position.

Relevant Facts (Semantic Memory):
[{'node': {'type': 'User', 'id': 'Alex', 'properties': {}}, 'r': {'type': 'HAS_GOAL', 'properties': {}}, 'related_node': {'type': 'InvestmentPhilosophy', 'id': 'Conservative Investing', 'properties': {}}}, {'node': {'type': 'User', 'id': 'Alex', 'properties': {}}, 'r': {'type': 'INTERESTED_IN', 'properties': {}}, 'related_node': {'type': 'Sector', 'id': 'Tech', 'properties': {}}}]
--- Generating Response ---
Generated Response:
Of course. Based on your stated goal of conservative investing in the tech sector, a great alternative to Apple (AAPL) would be Microsoft (MSFT).

Here's why it fits your profile

### Инспекция Хранилищ Памяти

Давайте заглянем под капот. Мы можем сделать запросы к нашим базам данных напрямую, чтобы увидеть воспоминания, которые создал наш агент.

In [ ]:
console.print("--- 🔍 Инспекция Эпизодической памяти (Vector Store) ---")
# Мы сделаем поиск по сходству для общего понятия, чтобы увидеть, что найдется
retrieved_docs = episodic_vector_store.similarity_search("User's investment strategy", k=3)
for i, doc in enumerate(retrieved_docs):
    print(f"{i+1}. {doc.page_content}")

console.print("\n--- 🕸️ Инспекция Семантической памяти (Graph Database) ---")
print(f"Graph Schema:\n{graph.get_schema}")

# Cypher-запрос, чтобы увидеть, кто чем интересуется
query_result = graph.query("MATCH (n:User)-[r:INTERESTED_IN|HAS_GOAL]->(m) RETURN n, r, m")
print(f"Relationships in Graph:\n{query_result}")

--- 🔍 Inspecting Episodic Memory (Vector Store) ---


1. The user, Alex, introduced himself as a conservative investor interested in established tech companies.
2. Based on the user's conservative investment goals, the assistant suggested Microsoft (MSFT) as a good alternative to Apple (AAPL), highlighting its diversification and enterprise focus.
3. The user inquired about Apple (AAPL), and the assistant confirmed it's a suitable stock for conservative investors due to its stability and market position.



--- 🕸️ Inspecting Semantic Memory (Graph Database) ---


Graph Schema:
{'node_props': {'InvestmentPhilosophy': [{'property': 'id', 'type': 'STRING'}], 'Company': [{'property': 'id', 'type': 'STRING'}], 'Sector': [{'property': 'id', 'type': 'STRING'}], 'User': [{'property': 'id', 'type': 'STRING'}]}, 'rel_props': {}, 'relationships': [{'start': 'User', 'type': 'INTERESTED_IN', 'end': 'Sector'}, {'start': 'User', 'type': 'INTERESTED_IN', 'end': 'Company'}, {'start': 'User', 'type': 'HAS_GOAL', 'end': 'InvestmentPhilosophy'}]}

Relationships in Graph:
[{'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'Conservative Investing', 'type': 'InvestmentPhilosophy'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'Tech', 'type': 'Sector'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'AAPL', 'type': 'Company'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'MSFT', 'type': 'Company'}}]


## Заключение

В этом блокноте мы успешно построили агента с сложной системой долгосрочной памяти. Демонстрация ясно показывает мощь этой архитектуры:

- **Провал без состояния:** Стандартный агент, если его спросить "Основываясь на моих целях, какая есть хорошая альтернатива?", потерпел бы неудачу, потому что у него нет памяти о целях пользователя.
- **Успех с дополненной памятью:** Наш агент преуспел, потому что он смог:
    1.  **Вспомнить эпизодически:** Он извлек резюме первого разговора: "Пользователь, Алекс, представился как консервативный инвестор..."
    2.  **Вспомнить семантически:** Он запросил граф и нашел структурированный факт: `(User: Alex) -[HAS_GOAL]-> (InvestmentPhilosophy: Conservative)`.
    3.  **Синтезировать:** Он использовал этот комбинированный контекст, чтобы предоставить высокорелевантную и персонализированную рекомендацию (Microsoft), явно ссылаясь на консервативные цели пользователя.

Эта комбинация воспоминания о том, *что произошло* (эпизодическая), и того, *что известно* (семантическая), является мощной парадигмой для перехода от простых, транзакционных агентов к созданию настоящих, обучающихся компаньонов. Хотя управление этой памятью в масштабе представляет собой сложности, такие как очистка и консолидация, фундаментальная архитектура, которую мы здесь построили, является значительным шагом к более интеллектуальным и персонализированным системам ИИ.